Code taken from "For Saving Statistics Data" on 8/11/2022

## Goal

Create a learning sequence of multiple NREM and REM phases, making the number of NREM+REM cycles a variable that is easily changed. Also include a parameter for a gap of time between when one backbone turns off and the other activates.



## Changes




## Notes

In [1]:
import numpy as np
import scipy
import scipy.stats as stats
import matplotlib.pyplot as plt
import random as RD
from matplotlib import colors
import winsound
import csv
import copy
from tqdm import tqdm
import seaborn as sns
import pickle

In [2]:
class parameters:
    
    def __init__(self):
        self.numEquations = 4
        self.stepSize = 0.1
        self.simLength = 42000
        self.tarray = np.arange(0,self.simLength,self.stepSize)
        self.Ntimes = len(self.tarray)
        self.spikeThreshold = 5 # Sets the voltage(mV) at which a spike is recorded. 
        self.numnrn = 180 # Number of neurons in the model.
        self.numSST = 20 # Number of SST neurons to be forced into the model.
        self.c_e = 0.1 # Percent connectivity for excitatory neurons.
        self.c_i = 0.5 # Percent connectivity for inhibitory neurons.
        self.p_e = 0.8 # Probabilty of an existing connection breaking and forming a new connection, excitatory.
        self.p_i = 0 # Probabilty of an existing connection breaking and forming a new connection, inhibitory.
        self.local_conn = True # When true, new connections can be formed with local connections. When false, only non-local new
                        # connections are formed.
            
        self.gks_NREM = 1.5 # ACh level of neurons during NREM phase. Type I is gks=0, Type II gks=1.5.
        self.gks_REM = 0 # ACh level for neurons during REM phase.
        self.gks_test = 0.1 # ACh level for neurons during testing phases.
        self.Idrive_min = 0.5 # Lower range for possible random Idrives. For excitory neurons.
        self.Idrive_max = 0.5 # Upper range for possible random Idrives. For excitory neurons.
        self.Idrive_SST = -0.1 #The Idrive for SST neurons. Should be low enough so that they do not fire without synaptic input. 
        self.Idrive_LE = -6 # Idrive to be assigned to LE neurons (or a subset of them).
        self.Idrive_NABB = -6 # Idrive for the non-active backbone, should be low enough to prevent any firing outside of noise.

        self.w_max = 5 #Maximum positive synaptic plasticity multiplier allowed in network.
        
        self.NABP_boo = True # When true, non-active backbone has plasticity. When False, it does not. 
        self.makeSound = True # Determines whether to play the three tones after simulation is finished.
        self.plas_skip_time = 200 # Time before spikes start being recorded (in ms) and plasticity begins.
        self.NREMtest_boo = False # Currently not used anywhere. Needs code needs to be added to make the NREM testing phase optional.
        self.RD_seed = True # When true, a seed is used to generate connections
        self.sim_seed = 4 # The seed for generating all random elements of the simulation. NOTE: defining a seed before a
                        # sequence of random events will not only define the outcome of the first random choice/event, but
                        # also the following ones. So we only need one seed.
        self.bbs_toplot = [1,2] # list of backbones to plot by ID.
        self.directory = r'' # The directory for all files to be saved to.
        
        self.osc_period = 200 # The amount of time each BB is active in the oscillations. Note that param.osc_period under 1000*param.plas_thr will
                        # likely cause large depotentiation problems. Also, we need param.osc_period < 1000/param.dep_thr to 
                        # prevent large depotentiation.
        self.BB_act_gap = 0 # The time (in ms) between when one backbone turns off and the other activates in learning.
                
        self.num_lrn_cycles = 2 # The number of NREM+REM learning cycles for the simulation.
        self.num_test_phases = 1 + self.num_lrn_cycles # Number of test phases in simulation.
        self.BB_len_test = 1500 # Length of test phase for each BB (in ms). Total test phase length is then 2*BB_len_test.
        self.len_NREM = int((self.simLength - 2*self.BB_len_test*self.num_test_phases)/(2*self.num_lrn_cycles)) # Length of NREM for both BBs for 1 learning cycle.
        self.len_REM = int((self.simLength - 2*self.BB_len_test*self.num_test_phases)/(2*self.num_lrn_cycles)) # Length of REM for both BBs for 1 learning cycle.
        self.t_start_posttest = self.simLength - 2*self.BB_len_test # The time at which the final testing phase begins.
        
        self.storage_freq = 20 # How often (in ms) to store connection weight data for each neuron. A value of 10 collects data every 
                        # 10 ms. At 1 ms, I was having memory issues on Michal's PC because the arrays were getting too large.
        
        self.w_EE = 0.15 # AMPA connection strength excitatory to excitatory.
        self.w_EI = 0.08 # AMPA connection strength excitatory to inhibitory.
        self.w_II = 0.15 # GABA A connection strength inhibitory to inhibitory.
        self.w_IE = 0 # GABA A connection strength inhibitory to excitatory.
        self.w_II_B = 0 # GABA B connection strength inhibitory to inhibitory.
        self.w_IE_B = 0.05 # GABA B connection strength inhibitory to excitatory.
        
        self.LEtoBB_mult = 1 # Multiplier for connections from LE to BB neurons.
        self.LEtoLE_mult = 1 # Multiplier for connections from LE to LE neurons.
        self.LEtoSST_mult = 1 # Multiplier for connections from LE to SST neurons.
        self.BBtoLE_mult = 1 # Multiplier for connections from BB to LE neurons.
        self.SSTtoLE_mult = 3.5 # Multiplier for connections from SST to LE neurons.
        self.SSTtoBB_mult = 0.5 # Multiplier for connections from SST to BB neurons.
        
        self.A_dep = 0.025 # The maximum amount a synapse can depotentiate per spike.
        self.A_pot = 0.07 # The maximum amount a synapse can potentiate per spike.
        self.tau_dep = 34 # Time constant for depotentiation side of STDP rule.
        self.tau_pot = 14 # Time constant for potentiation side of STDP rule.
        self.const_ISI = 30 # The inter-spike interval (in ms) at which to freeze the magnitude of the depotentiation portion
        # of the STDP rule so that it stays constant for larger ISIs.
        
        self.LEtoLE_plas_mult = 0.3 # Multiplier to modify the plasticity rate of LE to LE connections.
        self.LEtoBB_plas_mult = 0.3 # Multiplier to modify the plasticity rate of LE to BB connections.

        self.bg_str = 0.5 # The amount to strengthen connections from BB to blue and green LE neurons.
        self.pu_weak = 0 # The amount to weaken connections from both BBs to purple LE neurons.
        self.pi_weak = 0 # The amount to weaken connections from both BBs to pink LE neurons.
        
        self.gt_list = [] # List to hold the times and values for switching ACh levels during the simulation. [[gks val, time(ms)],[...]].
        self.bbt_list = [] # List to hold the times for switching BB activity. [ [ ['on',[bbs to turn on],'off',[bbs to turn off] ], time(ms)] , [...] ].
        self.plast_list = [] # List to hold the specific arangment and times to impliment network plasticity. [ [[LE,bb1,bb2],time] , ... ].
            

def init_param(): # Function to simply initialize a parameter object. Desired changes to parameters should be made here,
    # as opposed to being made in the parameter class function (except changes that affect definitions within the parameter
    # class definition).

    param = parameters()
    
    # The directory must be updated to have forward slashes, including a forward slash on the end.
    param.directory = param.directory.replace('\\','/') # Replaces backslash "\" with forward slash "/".
    param.directory = param.directory + '/' # Adds forward slash to end of string.
    
    
    # Sets up gks changing during simulation.
    cycle_time = 0 # The start time of the current cycle. Used to hold the time for reference when appending to gt_list.
    for cycle in range(param.num_lrn_cycles):
        
        param.gt_list.append([param.gks_test, cycle_time]) # Adds test phase.
        param.gt_list.append([param.gks_NREM, cycle_time + 2*param.BB_len_test]) # Adds NREM.
        param.gt_list.append([param.gks_REM, cycle_time + 2*param.BB_len_test + param.len_NREM]) # Adds REM.
        
        cycle_time = cycle_time + 2*param.BB_len_test + param.len_NREM + param.len_REM # Updates cycle_time.
    
    param.gt_list.append([param.gks_test, param.t_start_posttest]) # Adds final testing phase.
    
    
    
    # Sets up BB activity changing during simulation.
    for gks_val, time in param.gt_list: # Uses gt_list to assign BB activity times. Excludes first element because this is added manually above.
        
        if gks_val == param.gks_test:
            param.bbt_list.append([['on',[1],'off',[2]], time])
            param.bbt_list.append([['on',[2],'off',[1]], time + param.BB_len_test])
            
            
        if gks_val == param.gks_NREM:
            
            for osc_t in np.arange(time, time + param.len_NREM, param.osc_period + param.BB_act_gap):
                # Note that because this calls the last value of bbt_list, it is necessary to have a testing phase come first 
                # so that at least one entry is added to bbt_list before the following code.

                param.bbt_list.append([['on',[],'off',[1,2]], osc_t])
                    
                # The following ensures that the bbs switch in actvity.
                if param.bbt_list[-2][0] == ['on',[1],'off',[2]] and param.bbt_list[-1][0] == ['on',[],'off',[1,2]]: 
                    param.bbt_list.append([['on',[2],'off',[1]], osc_t + param.BB_act_gap])

                elif param.bbt_list[-2][0] == ['on',[2],'off',[1]] and param.bbt_list[-1][0] == ['on',[],'off',[1,2]]:
                    param.bbt_list.append([['on',[1],'off',[2]], osc_t + param.BB_act_gap])
                    
     
        if gks_val == param.gks_REM:
            
            for osc_t in np.arange(time, time + param.len_REM, param.osc_period + param.BB_act_gap):
                
                param.bbt_list.append([['on',[],'off',[1,2]], osc_t])
                
                if param.bbt_list[-2][0] == ['on',[1],'off',[2]] and param.bbt_list[-1][0] == ['on',[],'off',[1,2]]: 
                    param.bbt_list.append([['on',[2],'off',[1]], osc_t + param.BB_act_gap])

                elif param.bbt_list[-2][0] == ['on',[2],'off',[1]] and param.bbt_list[-1][0] == ['on',[],'off',[1,2]]:
                    param.bbt_list.append([['on',[1],'off',[2]], osc_t + param.BB_act_gap])
                    
               
       
    
    # Setting up network plasticity changes during simulation:
    for gks_val, time in param.gt_list:
        # Turns on plasticity for LE, BB1 and BB2 at all times except during testing phases.
        if gks_val == param.gks_test:
            param.plast_list.append([[False,False,False], time])
            
        if gks_val == param.gks_NREM or gks_val == param.gks_REM:
            param.plast_list.append([[True,True,True], time])
    
    
    
#     print('gt_list: ', param.gt_list)
#     print('bbt_list: ', param.bbt_list)
#     print(param.plast_list)
    
    return param


        

class neuron:
        
    def __init__(self):
        self.ID = 0
        self.position = []
        self.connections = [] #List of (1 or 0) connection strengths to other neurons. Is 2D list like [[postsyn,conn],[postsyn,conn]...].
        # DO NOT CHANGE self.connections from values of only 1 and 0 because many apects of the program rely on it.
        self.connectionWeights = [] #Holds changes made from plasticity. For self as presynaptic nrn. Values are strengths of
        # signal to other neurons from this neuron.
        self.Input_syn = 0
        self.Input_noise = 0
        self.Input_external = 0
        self.spikeTimes = [] # Set to record a spike when membrane voltage breaches variable param.spikeThreshold.
        self.prevActivity = 0
        self.neuronsInRange = [] #Tracks the # of neurons in range so as to minimize looping time during connection growth function
        self.solutions = np.zeros(param.numEquations) #Why does nrn.solutions still function as a comment?
        
        #Things I have added in myself:
        self.gks = param.gks_test # gks value for neuron, determines effective ACh concentration.
        self.spike = False #Determines whether the neuron has already spiked or not. 
        self.Idrive = 0
        self.color = '' #Color of neuron for graphing.
        self.conn_in = [] #Connections coming in from other neurons. Sum is the in-degree of the neuron. Note, not tuple like self.connections.
        # Format is 1D list of connection strengths, where list index is presynaptic neuron. 
        self.category = 'Excitatory' #Labels the neuron type. Default is excitatory, can be chanegd to inhibitory. 
        self.gsyn = 1 # Connection strength multiplier for I->E connections. ------- As of 6/11/22 gsyn is non-functional. It now
        # ony is used by the function sort_gsyn() for sorting BB and LE neurons. I should change code to sort based on backbone_ID.
        self.pair_spiketimes = np.zeros(param.numnrn) #Pair spike times for outgoing connections. Note that this only holds the most recent pair spiketime for each conn.
        self.start_noise = 0 #Starting step time for noise when it occurs (mV/param.stepSize). 
        self.backbone_ID = 0 # backbone_ID=0 will be used to designate lower E neurons and -1 for inhibitory neurons.
        self.spike_gaussian = [] #List of gaussian curves, each centered at the time a neuron spikes. Each index in list corresponds to a t_ind time.
        self.plas_on = True #Boolean determining whether or not to change plasticity of connections TO and FROM this nrn. 
        self.cw_in_history = [] #Connection weight history. Holds plasticity connection weights coming IN (this nrn as postsyn).
        # Set up as [[[weight from nrn 0, weight from nrn 1, ... ],time(ms)] ,...], one weight list for each milisecond.
        # List set up to skip first plas_skip_time ms because we don't want plasticity due to transient behaviors. Note the default
        # value for connections and non-existent connections is 1. 
        self.cw_out_history = [] # Same as cw_in_history, just with this nrn as the presynaptic neuron.
        self.scatter_color = 'grey'
        self.quad_color = 'grey' # Color assigned to LE neurons based on initial connectivity to BBs.

        
        
def equations(solns_, eqn, Isyn, Idrive,nrn):
    
    
    tempVal = 0
    
    category = nrn.category
    Inoise = nrn.Input_noise # Noise from neuron. Maybe I should put Idrive, solns, etc here as well?
    
    if category == 'Excitatory':
        gks = nrn.gks
        C = 1 
        gna = 24
        gkdr = 3
        gl = 0.02
        Vna = 55
        Vk = -90
        Vl = -60
        if(eqn == 0):
            hinf = 1/(1+np.exp((solns_[3]+53)/7))
            tauh = .37 + 2.78/(1+np.exp((solns_[3]+40.5)/6))
            tempVal = (hinf - solns_[0])/tauh 
        elif(eqn == 1):
            ninf = 1/(1+np.exp((-solns_[3]-30)/10))
            taun = .37 + 1.85/(1+np.exp((solns_[3]+27)/15))
            tempVal = (ninf - solns_[1])/taun
        elif(eqn == 2):
            zinf = 1/(1+np.exp((-solns_[3]-39)/5))
            tempVal = (zinf - solns_[2])/75
        elif(eqn == 3):
            m = 1/(1+np.exp((-solns_[3]-30)/9.5))
            tempVal = (-gna*(m**3)*solns_[0]*(solns_[3]-Vna) - gkdr*(solns_[1]**4)*(solns_[3]-Vk) 
                       - gks*solns_[2]*(solns_[3]-Vk) - gl*(solns_[3]-Vl) + Idrive - Isyn + Inoise)/C
            
    elif category == 'SST':
        gks = nrn.gks
        C = 1 
        gna = 24
        gkdr = 3
        gl = 0.02
        Vna = 55
        Vk = -90
        Vl = -60
        if(eqn == 0):
            hinf = 1/(1+np.exp((solns_[3]+53)/7))
            tauh = .37 + 2.78/(1+np.exp((solns_[3]+40.5)/6))
            tempVal = (hinf - solns_[0])/tauh 
        elif(eqn == 1):
            ninf = 1/(1+np.exp((-solns_[3]-30)/10))
            taun = .37 + 1.85/(1+np.exp((solns_[3]+27)/15))
            tempVal = (ninf - solns_[1])/taun
        elif(eqn == 2):
            zinf = 1/(1+np.exp((-solns_[3]-39)/5))
            tempVal = (zinf - solns_[2])/75
        elif(eqn == 3):
            m = 1/(1+np.exp((-solns_[3]-30)/9.5))
            tempVal = (-gna*(m**3)*solns_[0]*(solns_[3]-Vna) - gkdr*(solns_[1]**4)*(solns_[3]-Vk) 
                       - gks*solns_[2]*(solns_[3]-Vk) - gl*(solns_[3]-Vl) + Idrive - Isyn)/C
            
    elif category == 'PV+':
        gks = nrn.gks
        C = 1 
        gna = 35
        gkdr = 9
        gl = 0.1
        Vna = 55
        Vk = -90
        Vl = -65    
        if(eqn == 0):
            a_h = 0.07*np.exp(-(solns_[3]+58)/20)
            b_h = 1/(np.exp(-0.1*(solns_[3]+28))+1)
            phi = 5
            tempVal = phi*(a_h*(1-solns_[0]) - b_h*solns_[0])
        elif(eqn == 1):
            a_n = -0.01*(solns_[3]+34)/(np.exp(-0.1*(solns_[3]+34))-1)
            b_n = 0.125*np.exp(-(solns_[3]+44)/80)
            phi = 5
            tempVal = phi*(a_n*(1-solns_[1])-b_n*solns_[1])
        elif(eqn == 2):
            zinf = 1/(1+np.exp((-solns_[3]-39)/5))
            tempVal = (zinf - solns_[2])/75
        elif(eqn == 3):
            a_m = -0.1*(solns_[3]+35)/(np.exp(-0.1*(solns_[3]+35))-1)
            b_m = 4*np.exp(-(solns_[3]+60)/18)
            m = a_m/(a_m+b_m)
            tempVal = (-gna*(m**3)*solns_[0]*(solns_[3]-Vna) - gkdr*(solns_[1]**4)*(solns_[3]-Vk) 
                       - gks*solns_[2]*(solns_[3]-Vk) - gl*(solns_[3]-Vl) + Idrive - Isyn)/C
        
    return tempVal


def RK4(t_ind):
    
    global neuron
    
    for nrn in neurons:
        
        
        solns = nrn.solutions
        Isyn = nrn.Input_syn
        Idrive = nrn.Idrive
        k1 = np.zeros(param.numEquations)
        k2 = np.zeros(param.numEquations)
        k3 = np.zeros(param.numEquations)
        k4 = np.zeros(param.numEquations)
        
        init_solns = solns
        
        #Calculates the k1 variables
        for ii in range(len(solns)):
            k1[ii] = param.stepSize*equations(solns, ii,Isyn,Idrive,nrn)

        #Calculates the k2 variables
        for ii in range(len(solns)):
            k2[ii] = param.stepSize*equations(solns+k1/2, ii,Isyn,Idrive,nrn) #important fix done here. solns must be advanced by k
                                                                    #for calculation of the next k variable.
        #Calculates the k3 variables
        for ii in range(len(solns)):
            k3[ii] = param.stepSize*equations(solns+k2/2, ii,Isyn,Idrive,nrn) 

        #Calculates the k4 variables
        for ii in range(len(solns)):
            k4[ii] = param.stepSize*equations(solns+k3, ii,Isyn,Idrive,nrn)
        
        #Updates the general solution
        for ii in range(len(solns)):
            solns[ii] = init_solns[ii] + (k1[ii] + 2*k2[ii] + 2*k3[ii] + k4[ii])/6 
            nrn.solutions[ii] = solns[ii]
            
            
            
def init_nrn(): #initializes neurons and assigns ID, connections, weights, etc. 
    global neuron
    neurons = [] #List containing neuron objects
    nconn_Mat = [np.empty(3)] # 2D matrix for storing new connections.
    
    if param.RD_seed: # When true, the simulation will be reproducable entirely (all connections, neuron assignments, initial coniditions).
        RD.seed(param.sim_seed)
    
    def count_SST(neurons): # A function for counting the number of SST neurons.
        count = 0
        for nrn in neurons:
            if nrn.category == 'SST':
                count += 1
        return count

    
    for i in range(param.numnrn):  
        neurons = np.append(neurons,neuron()) #Intiallizes param.numnrn number of neurons
        
        
    #This for loop ensures that exactly param.numSST number of E neurons are changed to SST.
    for i in range(param.numSST):
        changed_to_SST = False #Keeps loop running until excitatory neuron is found to change to SST neuron.
        while changed_to_SST == False: #Loop mentioned above.
            nrn = RD.choice(neurons) #grabs one neuron object at random (available for editing)
            if nrn.category == 'Excitatory': #If true, turns excitatory neuron to SST. If neuron is not Excitatory, while loop runs again.
                nrn.category = 'SST'
                nrn.backbone_ID = -1 #Assigns inhibitory neurons to backbone ID = -1.
                changed_to_SST = True
                
    #Create list of only E neurons.
    Eneurons = []
    for nrn in neurons:
        if nrn.category == 'Excitatory':
            Eneurons.append(nrn) #Note that even though this is a different list than neurons, the neuron objects within can be
            # changed all the same like they were in neurons. 
                
                
    #Changes all excitatory neurons to having high inhibition (i.e. like LE neurons):
    for nrn in Eneurons:
        nrn.gsyn = 3
    
    
    
    def create_backbones(Eneurons):
        #this function initializes backbones into a network assuming that all E neurons have a high gysn (high inhibition level). 
        # Takes list of excitatory neuron objects as input. WILL NEED TO FIX WITH try->except when num_per_bb > non_bb_size because
        #then LElist will sample too many elements from templist on last run of loop. 
        global neuron 

        num_bb = 2 #number of backbones to create from number of available E neurons
        non_bb_size = 80 #number of non-backbone E neurons to be left in network.
        num_bb_nrns = param.numnrn - (param.numSST + non_bb_size) #number of neurons for splitting into backbones.
        num_per_bb = int(num_bb_nrns/num_bb) # number of E neurons per backbone.

        bb_list = RD.sample(Eneurons,num_per_bb) #temp list for looping. Randomly samples num_per_bb # of neurons from Eneurons.

        for bb in range(1,num_bb+1): #This makes the notation easier by shifitng indicies +1. This is because nrn.backbone_ID=0
        # is reserved for NON-backbone neurons. bb=1 designates backbone_ID=1.
            templist=[]

            for nrn in bb_list:
                nrn.backbone_ID = bb # Assigns first randomly chosen group of neurons to the first backbone.
                nrn.gsyn = 0.5 #Backbones are created in the "on" state.
                
            for nrn in Eneurons: #Makes a new list with only non-backbone nrns.
                if nrn.backbone_ID == 0:
                    templist.append(nrn)
            bb_list = RD.sample(templist,num_per_bb)

        
        
    create_backbones(Eneurons)


    ID = 0
    bb_colors = ['cyan','blue','green','orange','purple'] # Colors for backbones, cyan reserved for non-backbone E neurons.
    for nrn in neurons: #assigns neurons in list their IDs, init voltage, Idrive, etc.
        nrn.ID = ID
        ID += 1 
        nrn.spikeTimes = []
        nrn.solutions = [RD.random(),RD.random(),RD.random(),RD.uniform(-55,-20)] #Initial conditions of each neuron. Initial voltage randomly assigned between -55 and -20 mV.
        nrn.connectionWeights = [1]*param.numnrn #Creates a list of all connection weights to other neurons at value 1. 

        if nrn.category == 'Excitatory':
            nrn.Idrive = round(RD.uniform(param.Idrive_min, param.Idrive_max),3) #Random value between min and max rounded to 1 decimal places
            nrn.color = bb_colors[nrn.backbone_ID] #Assigns nrn color coded for backbone.
            
        if nrn.category == 'SST':
            nrn.color = 'Red' #Inhibitory given red.
            nrn.Idrive = param.Idrive_SST #Idrive for inhibitory neurons. 
        if nrn.category == 'PV+':
            nrn.color = 'darkorange'
            nrn.Idrive = round(RD.uniform(Idrive_PVplus_min,Idrive_PVplus_max),3)
    
    conn_Matrix = np.zeros((param.numnrn,param.numnrn)) #initializes matrix of zeros with param.numnrn x param.numnrn size. Row = nrn #, Column = connected nrn #
    # Fills matrix with connectivity based on proximity. conn_span # of neurons to right and left are given full connection. 
    for row_index, row in enumerate(conn_Matrix):
        for column_index, conn in enumerate(row):
            
            if neurons[row_index].category == 'Excitatory': #Determines which connectivity percent to use based on neuron category.
                conn_span = int(param.c_e*param.numnrn/2) #number of neurons to be connected on either side of a neuron.

                #sets neurons at +- conn_span from diagonal to full connectivity.
                if column_index >= row_index-conn_span and column_index <= row_index+conn_span:
                    conn = 1 
                #Full connectivity at edge case of first neurons connected to last neurons in ring.
                elif row_index-conn_span < 0 and column_index >= param.numnrn+row_index-conn_span:
                    conn = 1
                #Full connectivity at edge case of last neurons connected to first neurons in ring.
                elif row_index+conn_span > (param.numnrn-1) and column_index <= row_index-param.numnrn+conn_span:
                    conn = 1 
                #All other neurons have zero connectivity.
                else:
                    conn = 0
                # Sets diagonal entries to zero.
                if column_index == row_index:
                    conn = 0
                    
            elif neurons[row_index].category == 'SST' or neurons[row_index].category == 'PV+': # If the presynaptic neuron is inhibitory.
                if RD.random() <= param.c_i and column_index != row_index: # if a random between 0 and 1 is less than the connectivity percent. 
                    conn = 1
                else:
                    conn = 0
                    
            row[column_index] = conn  #Assigns the local connections.
        conn_Matrix[row_index] = row

    # Changes connections based on proability p. 
    for row_index, row in enumerate(conn_Matrix): 
        row_temp = row.copy() #used to store changes while deleting connections from new_conn_list. VERY IMPORTANT TO USE .copy()
         # otherwise row will change when row_temp is changed. This is how assignment works. 
        if neurons[row_index].category == 'Excitatory': #Determines which connectivity percent to use based on neuron category.
            conn_span = int(param.c_e*param.numnrn/2) #number of neurons to be connected on either side of a neuron.
            p = param.p_e
        elif neurons[row_index].category == 'SST' or neurons[row_index].category == 'PV+':
            conn_span = int(param.c_i*param.numnrn/2)
            p = param.p_i 

        for column_index, conn in enumerate(row):
            
            if conn != 0: #only for existing connections.
                if RD.random() <= p: # RD.random() selects random float between 0 and 1.

                    if param.local_conn == True: # Allows new local connections.
                        new_conn_list = np.append(np.arange(0,row_index,1),np.arange(row_index+1,param.numnrn,1)) #Creates list of
                        #all nrn IDs besides self.
                    if param.local_conn == False: #No new local connections.
                        #List of all nrns except local and self. Very gross and uses heaviside functions. May be simplifiable. 
                        new_conn_list = np.append(np.arange(param.numnrn - param.numnrn*np.heaviside(row_index-conn_span-1, 1)
                                    +(row_index+conn_span-param.numnrn+1)*np.heaviside(row_index+conn_span-param.numnrn,1),row_index-conn_span,1),
                                    np.arange(row_index+conn_span+1,(param.numnrn+row_index-conn_span)-
                                    (row_index-conn_span)*np.heaviside(row_index-conn_span, 1),1))
                     
                    for index, val in enumerate(row_temp):#Deletes established conns from new_conn_list, preventing double connections.

                        if val != 0: #Sorts out only established conns.
                            delindex = np.where(new_conn_list == index) #Finds where est. conn lies in new_conn_list.
                            if len(delindex[0]) > 0: #Stops error from having nothing to delete when param.local_conn = False. 
                                delindex = delindex[0][0] #grabs useful integer.
                                new_conn_list = np.delete(new_conn_list, delindex) #deletes from possible conns. 
    
                    nconn = RD.choice(new_conn_list) #Randomly selects one neuron to connect to. 
                    nconn_info = [[row_index, column_index, nconn]] # [neuron #, old connection, new connection]. Must be 2D.
                    nconn_Mat = np.concatenate((nconn_Mat,nconn_info)) #Adds this info to a matrix for later use.
                    
                    #Updates values of the array used in determining new connections. 
                    row_temp[int(column_index)] = 0 
                    row_temp[int(nconn)] = 1


    nconn_Mat = np.delete(nconn_Mat,0,0) #Removes np.empty dummy row from matrix.
    
    #Apply new connection changes.
    for info in nconn_Mat:
        conn_Matrix[int(info[0]),int(info[1])] = 0 #Sets old connection to zero.
        conn_Matrix[int(info[0]),int(info[2])] += 1 #Establishes connection or adds another connection.

    nc_Matrix = np.empty((param.numnrn,param.numnrn,2)) #Empty matrix to hold final values. nc means neuron # and connection strength. 
    count = 0
    # Creates 3D array, nc_matrix, storing (nrn #, conn strength to nrn receiving Isyn)
    for row in conn_Matrix:
        conn_tuple = list(enumerate(row)) #list of tuples with info (postsyn nrn #, recieving nrn conn strength)
        nc_Matrix[count] = conn_tuple 
        count += 1

    
    #Assigns neuron objects the list of tuple connections. 
    for nrn in neurons:
        nrn.connections = nc_Matrix[nrn.ID] #Outgoing connections for nrn.
        nrn.conn_in = nc_Matrix[:,nrn.ID][:,1]# Incoming connections for nrn. [0,0,1,1] would mean this neuron recieves no
        # signal from neurons 0 and 1, and full signal from neurons 2 and 3. 
        
        
    def del_crossbb_conns():
        # This function deletes E-E connections between neurons in different backbones. This prevents the activity of one backbone
        # form exciting the other.
        global neuron

        for nrn1 in neurons:
            for nrn2 in neurons:
                #The following checks that nrns are not inhibitory (bb=-1) or lower excitatory (bb=0) and are in different backbones with a connection.
                if nrn1.backbone_ID not in [-1,0] and nrn2.backbone_ID not in [-1,0] and nrn1.backbone_ID != nrn2.backbone_ID and nrn1.connections[int(nrn2.ID)][1] == 1:
                    nrn1.connections[int(nrn2.ID)][1] = 0 #Eliminates connection between neurons.
                
    del_crossbb_conns()
    
    def del_LEtobb_conns():
        # This function deletes LE to BB connections. NOTE that the conn mat/ plas mat are not changed, so they still show these connections as existing.
        global neuron

        for nrn in neurons:
            for postsyn, conn in nrn.connections:
                if nrn.backbone_ID == 0 and neurons[int(postsyn)].backbone_ID in param.bbs_toplot:
                    nrn.connections[int(postsyn)][1] = 0 #Eliminates connection between neurons.

    #del_LEtobb_conns()
    
    
    return neurons,nc_Matrix




def init_quad_colors(): # Function for initializing the color groups of LE neurons based on their initial connections to bbs.
    # This function should be run after connections are established (after init_neurons()) but before the t_ind loop for the 
    # simulation begins.
    
    LE_neurons = []
    for nrn in neurons:
        if nrn.backbone_ID == 0:
            LE_neurons.append(nrn)
            
    quad_colors = ['blue', 'green', 'purple', 'pink']
    quad_counts = [0,0,0,0] # counts the number of LE assigned to each color. [blue, green, purple, pink].
    
    LE_sum_cw_bb1,LE_sum_cw_bb2 = np.zeros(len(LE_neurons)),np.zeros(len(LE_neurons)) #Arrays of cw sums for plotting on scatter plot.

    # Need to fill LE_sum_cw_bbx with number of connections (effectively the cw) from bbx. This will be used to sort for quadrant colors.
    # LE_sum_cw_bbx is analogous to bbx_REM_vals from BBs_scatter().
    for LE_nrn in LE_neurons:
        for index,conn in enumerate(LE_nrn.conn_in):
            if neurons[index].backbone_ID == 1 and conn == 1: # If presyn is bb1 and connection exists.
                LE_sum_cw_bb1[LE_neurons.index(LE_nrn)] += 1 # Adds 1 so that at after the loop, index represents total # of
                                                             # connections from bb1 to this LE nrn.   
            if neurons[index].backbone_ID == 2 and conn == 1:
                LE_sum_cw_bb2[LE_neurons.index(LE_nrn)] += 1
            
    # Zip all this data together with the neuron objects.
    nrns_and_cws = zip(LE_neurons, LE_sum_cw_bb1, LE_sum_cw_bb2)
    
    # First sort the list based on one of the backbones, say bb1. Strongest first in the list.
    sorted_bb1 = [(nrn,cw_bb1,cw_bb2) for nrn,cw_bb1,cw_bb2 in sorted(nrns_and_cws, reverse=True, key=sort_tuple_1)]
    
    # Then separate the sorted list into two halves, one half for the blue/purple neurons, the other for the green/pink.
    strong_bb1, weak_bb1 = [],[] 
    for index,val in enumerate(sorted_bb1):
        if index < len(LE_neurons)/2: 
            strong_bb1.append(val) # Sorts strongest bb1 connections into strong_bb1.
        else:
            weak_bb1.append(val) # Sorts weaker bb1 connections into weak_bb1.
    
    # Now I need to sort these two lists by strength of connections from the other backbone, bb2. Strongest first in the list.
    sorted_strong_bb1 = [(nrn,cw_bb1,cw_bb2) for nrn,cw_bb1,cw_bb2 in sorted(strong_bb1, reverse=True, key=sort_tuple_2)]
    sorted_weak_bb1 = [(nrn,cw_bb1,cw_bb2) for nrn,cw_bb1,cw_bb2 in sorted(weak_bb1, reverse=True, key=sort_tuple_2)]
    
    # Now everything should be fully sorted into four equal size groups based on connection strength to both backbones.
    # Remember that for each time the lists were sorted, the strongest connections were sorted to the beginning of the list.
    for index, (nrn,cw_bb1,cw_bb2) in enumerate(sorted_strong_bb1):
        
        if index < len(LE_neurons)/4: # Strongest to both bbs: purple LE neurons.
            nrn.quad_color = quad_colors[2]
            quad_counts[2] += 1
        else: # Strong to bb1 but weak to bb2: blue LE neurons.
            nrn.quad_color = quad_colors[0] 
            quad_counts[0] += 1    
            
    for index, (nrn,cw_bb1,cw_bb2) in enumerate(sorted_weak_bb1):
        
        if index < len(LE_neurons)/4: # Weak to bb1 but strong to bb2: green LE neurons.
            nrn.quad_color = quad_colors[1]
            quad_counts[1] += 1
        else: # Weak to both bbs: pink LE neurons.
            nrn.quad_color = quad_colors[3] 
            quad_counts[3] += 1   
    
#    print('Number LE Neurons. [blue, green, purple, pink]: ', quad_counts)
    
                    
                    
                    
            
    
            
def updateSyn(t_ind): #Gives synaptic input to all neurons on connection list
    #Includes changes in synaptic strengths. t_start is the time at which the presynaptic neuron's voltage breaches -20 mV.
    # Has been changed to normalize strength of inputs to a neuron by number of inputs. I.e sum of all inputs comes to param.w_max. 
    t_temp = 0 
    global neuron
    
    tau = 0.5 # Time constant for fast-acting receptors.
    tau_B = 50 # Time constant for GABA B receptors, slow-acting.
    
    
    for nrn in neurons:# presynaptic neurons.         
        if len(nrn.spikeTimes) > 0: # To prevent errors of calling [-1] from an array without any entries. Can change to be l > 2, 3 ...
            t_temp = nrn.spikeTimes[-1] #grabs time this neuron spikes at.

            for conn in nrn.connections: #Gives all postsynaptic neurons Isyn corrspondping to their voltage.
                if conn[1] != 0: #Prevents synaptic current from being calculated to non-connected neurons.
                    
                    V = neurons[int(conn[0])].solutions[3] #Voltage of postsynaptic neuron. Note conn[1] is the connection strength and conn[0] is the ID.
                    Isyn = 0 

                    if nrn.category == 'SST' or nrn.category == 'PV+': # Handles GABA A and B receptors in postsyn  neurons.
                        
                        E_syn = -75 # Chloride reversal potential. 
                        
                        if neurons[int(conn[0])].category == 'SST' or neurons[int(conn[0])].category == 'PV+': # For I-I connections.
                            for w,t in (param.w_II,tau),(param.w_II_B,tau_B): #Sends two signals, one with w_II/tau and one with param.w_II_B/tau_B. 
                                Isyn += conn[1]*(w)*np.exp(-param.stepSize*(t_ind-t_temp)/t)*(V - E_syn) # t is tau here. 
                                
                        if neurons[int(conn[0])].category == 'Excitatory': # For I->E connections.
                            
                            for w,t in (param.w_IE,tau),(param.w_IE_B,tau_B):
                                if neurons[int(conn[0])].backbone_ID == 0: # For I->LE connections.
                                    Isyn += param.SSTtoLE_mult*conn[1]*(w)*np.exp(-param.stepSize*(t_ind-t_temp)/t)*(V - E_syn)
                                else: # For other connections, i.e. I->BB
                                    Isyn += param.SSTtoBB_mult*conn[1]*(w)*np.exp(-param.stepSize*(t_ind-t_temp)/t)*(V - E_syn)
                                    
                    if nrn.category == 'Excitatory':
                        
                        E_syn = 0 # Sodium reversal potential. E_syn = 0 for excitory synapse and E_syn = -75 mV for inhibitory synapse
                        
                        if nrn.backbone_ID == 0: # If presynaptic neuron in an LE neuron.
                            if neurons[int(conn[0])].category == 'SST' or neurons[int(conn[0])].category == 'PV+': # For LE->I connections.
                                Isyn =param.LEtoSST_mult*conn[1]*(param.w_EI)*np.exp(-param.stepSize*(t_ind-t_temp)/tau)*(V - E_syn)
                            if neurons[int(conn[0])].category == 'Excitatory': # For LE-E connections.
                                if neurons[int(conn[0])].backbone_ID == 0: # If postsynaptic neuron is LE as well. 
                                    Isyn = param.LEtoLE_mult*nrn.connectionWeights[int(conn[0])]*conn[1]*(param.w_EE)*np.exp(-param.stepSize*(t_ind-t_temp)/tau)*(V - E_syn)
                                elif neurons[int(conn[0])].backbone_ID in param.bbs_toplot: # For LE->BB connections
                                    Isyn =param.LEtoBB_mult*nrn.connectionWeights[int(conn[0])]*conn[1]*(param.w_EE)*np.exp(-param.stepSize*(t_ind-t_temp)/tau)*(V - E_syn)
                        
                        else: # For non-LE presynaptic E neurons, i.e. BB neurons.
                            if neurons[int(conn[0])].category == 'SST' or neurons[int(conn[0])].category == 'PV+': # For BB->I connections.
                                Isyn = conn[1]*(param.w_EI)*np.exp(-param.stepSize*(t_ind-t_temp)/tau)*(V - E_syn)
                            if neurons[int(conn[0])].category == 'Excitatory': # For E-E connections.
                                if neurons[int(conn[0])].backbone_ID in param.bbs_toplot: # For BB->BB connections.
                                    Isyn = nrn.connectionWeights[int(conn[0])]*conn[1]*(param.w_EE)*np.exp(-param.stepSize*(t_ind-t_temp)/tau)*(V - E_syn)
                                if neurons[int(conn[0])].backbone_ID == 0: # For BB -> LE connections.
                                    Isyn = param.BBtoLE_mult*nrn.connectionWeights[int(conn[0])]*conn[1]*(param.w_EE)*np.exp(-param.stepSize*(t_ind-t_temp)/tau)*(V - E_syn)
                                    
                                    
                    neurons[int(conn[0])].Input_syn += Isyn #Isyn going to Postsynaptic neuron.
      
        #Additional portion of updateSyn() for calculating noise probabilities. INDENTATION IS IMPORTANT! MUST BE IN nrn LOOP.
        noise_mag = 80 #Magnitude of noise input. 
        probability = 2*10**(-4) #Probability in every integration step that a noise spike will occur for each neuron.
        t_noise = nrn.start_noise 
        global neuron

        if RD.random() <= probability: #Handles the start of a noise spike.
            #print('Noise Spike at Time ',t_ind*param.stepSize)
            Inoise = noise_mag
            nrn.start_noise = t_ind #Time at which noise begins 
        elif nrn.Input_noise != 0 and (t_ind - t_noise)*param.stepSize <= 1: #extends noise input for 1 ms.
            Inoise = noise_mag
        else:
            Inoise = 0 #When there is no noise. 

        nrn.Input_noise = Inoise # Applies noise to neuron. 

    
    
    
            
def updateSpikeTime(t_ind):
    global neuron
    # Variables for old, symmetric, frequency dependent synaptic plasticity rule:
#     A = 0.6/10 # The maximum amount a synapse can change in strength per spike.
#     tau = 10 # Time constant for exponential function in ms.

    # For new, unsymmetric plasticity rule. Depotentiation happens more weakly but over a wider time span, while potentiation 
    # happens stronly but only when the spikes are close together. This rule is also spike timing dependent, so pre->post is
    # always potentiated and post->pre always depotentiated. This plasticity rule also does not have a frequency dependent
    # component. For more details see "80 LE Network 5.ipynb". 


    for nrn in neurons:
        
        # Recording the incoming connection plasticity weights for each neuron:
        if t_ind % (param.storage_freq/param.stepSize) == 0 and t_ind*param.stepSize >= param.plas_skip_time: #Only runs code every ms (t_ind with only zero in decimal place). Also skips first 500 ms of no plasticity.
            
            presyn_weight_list = np.ones(int(param.numnrn)) #List for storing weight connections from each presyn neuron.
            postsyn_weight_list = np.ones(int(param.numnrn))
            
            for nrn_ID in range(param.numnrn): #Loops through presynaptic connections to nrn.
                presyn_weight_list[int(nrn_ID)] = neurons[int(nrn_ID)].connectionWeights[int(nrn.ID)] #Records value 
                # of connection weight from presynaptic neuron.
                postsyn_weight_list[int(nrn_ID)] = nrn.connectionWeights[int(nrn_ID)]
            
            nrn.cw_in_history.append([presyn_weight_list, t_ind*param.stepSize]) #Appends conn weights in to this nrn to history. 
            nrn.cw_out_history.append([postsyn_weight_list, t_ind*param.stepSize]) # Appends conn weights out of this nrn to history.

        # Recording spike times:
        if nrn.solutions[3] >= param.spikeThreshold and nrn.spike == False and (t_ind*param.stepSize) > param.plas_skip_time: #Selects spikes, skips anything before the first "param.plas_skip_time" ms. 
            
            nrn.spikeTimes = np.append(nrn.spikeTimes, t_ind) #Records (time/param.stepSize) of a spike.
            nrn.spike = True
                        
            if nrn.plas_on == True: #If nrn's plas_on is True, plasticity(to and from) nrn is allowed to change. If False, it is frozen.

                
                #Changes synaptic weights. Note, nrn.connections is a tuple [postsyn, conn] while nrn.conn_in is simply a list of conns where the index is the presyn.
                # This works using only spikes that have already occured, so in-conns will always be strengthened and out-conns always weakened.
                for postsyn,conn in nrn.connections: # The outgoing connections. This weakens synapses. 

                    if conn == 1 and nrn.category == 'Excitatory' and neurons[int(postsyn)].category == 'Excitatory' \
                    and len(neurons[int(postsyn)].spikeTimes) > 0 and neurons[int(postsyn)].plas_on == True: # Existing E-E conns for neurons that have spiked and have plas_on.
                        
                        # Excludes inter-BB conns and LE to BB conns from plasticity. Also excludes b-g LE.
                        if (nrn.backbone_ID == neurons[int(postsyn)].backbone_ID and nrn.backbone_ID in param.bbs_toplot):
#                         or (nrn.backbone_ID == 0 and neurons[int(postsyn)].backbone_ID == 0 and nrn.quad_color in\
#                            ['blue','green'] and neurons[int(postsyn)].quad_color in ['blue','green'] and\
#                            nrn.quad_color != neurons[int(postsyn)].quad_color): 
                            pass # Does nothing if the above is true.

                        else:
                            ISI = param.stepSize*abs(neurons[int(postsyn)].spikeTimes[-1] - nrn.spikeTimes[-1]) # The inter-spike interval in ms.
                            if ISI <= param.const_ISI: # Portion for the normal exponential STDP rule shape.
                                
                                if nrn.backbone_ID == 0 and neurons[int(postsyn)].backbone_ID == 0: # For LE-LE connections.
                                    nrn.connectionWeights[int(postsyn)] += param.LEtoLE_plas_mult*-param.A_dep*np.exp(-ISI/param.tau_dep) # Weaken LE-LE synapse.
                                
                                elif nrn.backbone_ID == 0 and neurons[int(postsyn)].backbone_ID in param.bbs_toplot: # LE to BB connections. 
                                    nrn.connectionWeights[int(postsyn)] += param.LEtoBB_plas_mult*-param.A_dep*np.exp(-ISI/param.tau_dep) # Weaken LE-BB synapse.
                                    
                                else: # For all other E-E connections.
                                    nrn.connectionWeights[int(postsyn)] += -param.A_dep*np.exp(-ISI/param.tau_dep) # Weaken synapse.
                            
                            elif ISI > param.const_ISI: # Portion for the flat line STDP rule shape. Value set to be the 
                                # value of the STDP rule when the ISI equals const_ISI. 
                                if nrn.backbone_ID == 0 and neurons[int(postsyn)].backbone_ID == 0: # For LE-LE connections.
                                    nrn.connectionWeights[int(postsyn)] += param.LEtoLE_plas_mult*-param.A_dep*np.exp(-param.const_ISI/param.tau_dep) # Weaken LE-LE synapse.
                                
                                elif nrn.backbone_ID == 0 and neurons[int(postsyn)].backbone_ID in param.bbs_toplot:
                                    nrn.connectionWeights[int(postsyn)] += param.LEtoBB_plas_mult*-param.A_dep*np.exp(-param.const_ISI/param.tau_dep)
                                
                                else: # For all other E-E connections.
                                    nrn.connectionWeights[int(postsyn)] += -param.A_dep*np.exp(-param.const_ISI/param.tau_dep) # Weaken synapse.
                                
                            if nrn.connectionWeights[int(postsyn)] < 0:
                                nrn.connectionWeights[int(postsyn)] = 0 #This prevents synaptic weakening below zero, which would simulate inhibition.
                  
                
                for presyn,conn in enumerate(nrn.conn_in): # Incoming connections. This strengthens synapses.

                    if conn == 1 and nrn.category == 'Excitatory' and neurons[int(presyn)].category == 'Excitatory' \
                    and len(neurons[int(presyn)].spikeTimes) > 0 and neurons[int(presyn)].plas_on == True:

                        if (nrn.backbone_ID == neurons[int(presyn)].backbone_ID and nrn.backbone_ID in param.bbs_toplot):
#                         or (nrn.backbone_ID == 0 and neurons[int(presyn)].backbone_ID == 0 and nrn.quad_color in\
#                            ['blue','green'] and neurons[int(presyn)].quad_color in ['blue','green'] and\
#                            nrn.quad_color != neurons[int(presyn)].quad_color):
                            pass 

                        else:
                            
                            if nrn.backbone_ID == 0 and neurons[int(presyn)].backbone_ID == 0: # LE-LE connections.
                                neurons[int(presyn)].connectionWeights[int(nrn.ID)] += param.LEtoLE_plas_mult*param.A_pot*np.exp((-param.stepSize*abs(neurons[int(presyn)].spikeTimes[-1] 
                                                                                            - nrn.spikeTimes[-1]))/param.tau_pot)# Weaken synapse.
                            
                            elif (nrn.backbone_ID in param.bbs_toplot and neurons[int(presyn)].backbone_ID == 0):
                                neurons[int(presyn)].connectionWeights[int(nrn.ID)] += param.LEtoBB_plas_mult*param.A_pot*np.exp((-param.stepSize*abs(neurons[int(presyn)].spikeTimes[-1] 
                                                                                            - nrn.spikeTimes[-1]))/param.tau_pot)
                            else: # All other E-E connections.
                                neurons[int(presyn)].connectionWeights[int(nrn.ID)] += param.A_pot*np.exp((-param.stepSize*abs(neurons[int(presyn)].spikeTimes[-1] 
                                                                                            - nrn.spikeTimes[-1]))/param.tau_pot)# Weaken synapse.
                            if neurons[int(presyn)].connectionWeights[int(nrn.ID)] > param.w_max:
                                neurons[int(presyn)].connectionWeights[int(nrn.ID)] = param.w_max # Caps strength at param.w_max.
                                        
        
        
        if nrn.solutions[3] <= -30 and nrn.spike == True: #Resets the spiking status, allows for next spike to be recorded. 
            nrn.spike = False
                                    
                                    


            
            
'---------------------------------------------------------------------------'      
def param_funcs():
    # Function for creating and returning lists needed in vary_param that don't need to be redone every timestep. 
    # NOTE: Before I actually implement this function, I need to test whether using it increases or decreases simulation runtime.
    dummy_var = 0


            
def vary_param(t_ind): #Changes gks of network as well as vary other parameters. I am also added in a component to weaken all synapses after NREM.

    global neuron
    
    # Vary gks:
    t = t_ind*param.stepSize #changes t_ind to ms.
    
    times = [i[1] for i in param.gt_list] # creates list of times from the gt_list.
    if t in times:
        gks_val = param.gt_list[times.index(t)][0] # assigns gks_val the corresponding gks value.
        
        for nrn in neurons:
            nrn.gks = gks_val # Changes the gks value to gks_val for all neurons.
        
        
        
    
    # Vary BB activity:           
    times_bb = [i[1] for i in param.bbt_list]

    if t in times_bb:
        #print('onoff triggered at',t)
        #'On' Operations:
        on_off = param.bbt_list[times_bb.index(t)][0][0] #This grabs the 'on' string
        bb_toswitch = param.bbt_list[times_bb.index(t)][0][1] #Grabs bb IDs to be turned on.
        bb_onoff(on_off,bb_toswitch)
        #'Off' Operations:
        on_off = param.bbt_list[times_bb.index(t)][0][2] #This grabs the 'off' string
        bb_toswitch = param.bbt_list[times_bb.index(t)][0][3] #Grabs bb IDs to be turned off.
        bb_onoff(on_off,bb_toswitch)
                
        
        
    # Vary plasticity:
    times_plas = [i[1] for i in param.plast_list]
    if t in times_plas: #If the simulation reaches one of desired times:
        plas_boo_list = param.plast_list[times_plas.index(t)][0] #Assigns plas_on the boolean associated with time t. 
        
        for nrn in neurons: #Changes plas_on attribute for all neurons to match plas_boo_list vals. Uses index. 
            nrn.plas_on = plas_boo_list[int(nrn.backbone_ID)]
        
        
'-------------------------------------------------------------------------'     
        
    
    
    
def zeroTempVars(): #Zeros all variables to prevent accidental accumulation of unwanted terms. Just a safety measure, good habit.
        #Note: Do not zero solutions, because they are used in calculating next solutions. 
        global neuron
        for nrn in neurons:
            nrn.Input_syn = 0 #Zeroed because ISyn must be added to account for input from multiple neurons. If Isyn was
            # simply assigned, the Isyn would not accumulate. But now it must be zeroed. 
            
            

    
    
def sort_gsyn(item): #Function for returning the gsyn of a neuron
    # Built to handle nrn simply as an object, and also as a list [nrn_object,index_val].
    if isinstance(item,tuple): #if item is a tuple.
        val = item[0].gsyn    
    elif isinstance(item,neurons[0].__class__): #If this item is object type of a neuron.
        val = item.gsyn
    elif isinstance(item, list): #If item is a list.
        val = item[0].gsyn
    else:
        print("Exception: val=0 in sort_gsyn.")
        val=0
    return val


def sort_Idrive(nrn):
    return nrn.Idrive


def sort_bb(nrn):
    val = nrn.backbone_ID
    return val

def sort_bb_tuple(list1):
    val = list1[0].backbone_ID
    return val
                    
def sort_tuple(tup): #Sorts tuple by first element (index 0).
    val = tup[0]
    return val

def sort_tuple_1(tup): # Sorts tuple by value at index 1.
    val = tup[1]
    return val

def sort_tuple_2(tup): # Sorts tuple by value at index 2.
    val = tup[2]
    return val

def sort_LE_bycolor(LE_nrn): # Function used to sort LE neurons by color. Green first, then blue, then pink, then purple.
    val = 0
    if LE_nrn.quad_color == 'green':
        val = 1
    if LE_nrn.quad_color == 'blue':
        val = 2
    if LE_nrn.quad_color == 'pink':
        val = 3
    if LE_nrn.quad_color == 'purple':
        val = 4
    
    return val
    
    
    

    
    
def beep(): #Makes a series of beeps. Meant to signal end of code-running. 
    if param.makeSound: #When True, plays beeps.
        winsound.Beep(349,500)
        winsound.Beep(440,500)
        winsound.Beep(523,500)
        
    
    

    
def strengthen_backbone(): # Increases strength of all connections between backbone neurons.
    global neuron
    for nrn1 in neurons: # Loops over all possible pairs of neurons.
        for nrn2 in neurons:
            if nrn1.backbone_ID == nrn2.backbone_ID and nrn1.backbone_ID not in [-1,0] and nrn1.connections[int(nrn2.ID)][1] == 1: #if both neurons are to be part of the same backbone and are actually connected.
                nrn1.connectionWeights[int(nrn2.ID)] = 2.5 # Strengthens connections between neurons in the same BB.
                
                
                
def strengthen_LE(): # Gives the LE neurons with largest number of connections from each backbone an increased connection weight
    # for all those connections, as well as a smaller connection weight for the least connected LE neurons. NOTE that this is 
    # based off of quad colors, which takes into account the number of connections from both bbs. So, in reality, the LE
    # neurons with most connections to one bb and also least connections to the other bb have their plasticity weights changed
    # for those connections. These are the blue and green quad color labeled LE neurons.
    # This function should be run after init_quad_colors() but before the t_ind loop for the simulation begins.
    global neuron
    
    LE_neurons = []
    for nrn in neurons:
        if nrn.backbone_ID == 0:
            LE_neurons.append(nrn)
            
    for nrn_LE in LE_neurons: # Changes plasticity weight based on quad color.
        for nrn in neurons:
            if nrn.connections[int(nrn_LE.ID)][1] == 1: # If connection from nrn to LE nrn exists.
                
                if nrn.backbone_ID == 1: # For connections from a bb1 neuron to LE neuron. 
                    if nrn_LE.quad_color == 'blue':
                        nrn.connectionWeights[int(nrn_LE.ID)] += param.bg_str # Strenghthens connections to LE neurons that have many
                        # synapses from bb1 and few synapses from bb2.
                    if nrn_LE.quad_color == 'green':
                        nrn.connectionWeights[int(nrn_LE.ID)] += 0 # Weakens connections to LE neurons that have many 
                        # synapses from bb2 and few synapses from bb1.
                    if nrn_LE.quad_color == 'purple': #Weakens connections to LE neurons strongly connected to both BBs.
                        nrn.connectionWeights[int(nrn_LE.ID)] += -param.pu_weak
                    if nrn_LE.quad_color == 'pink': #Weakens connections to LE neurons weakly connected to both BBs.
                        nrn.connectionWeights[int(nrn_LE.ID)] += -param.pi_weak
                        
                if nrn.backbone_ID == 2: # For connections from a bb2 neuron to LE neuron. 
                    if nrn_LE.quad_color == 'blue':
                        nrn.connectionWeights[int(nrn_LE.ID)] += 0 # Strenghthens connections to LE neurons that have many
                        # synapses from bb1 and few synapses from bb2.
                    if nrn_LE.quad_color == 'green':
                        nrn.connectionWeights[int(nrn_LE.ID)] += param.bg_str # Weakens connections to LE neurons that have many 
                        # synapses from bb2 and few synapses from bb1.
                    if nrn_LE.quad_color == 'purple': #Weakens connections to LE neurons strongly connected to both BBs.
                        nrn.connectionWeights[int(nrn_LE.ID)] += -param.pu_weak
                    if nrn_LE.quad_color == 'pink': #Weakens connections to LE neurons weakly connected to both BBs.
                        nrn.connectionWeights[int(nrn_LE.ID)] += -param.pi_weak
                        

        
        
        
    
    
def bb_onoff(onoff,bb_listtoswitch):
    # Function for turning backbones on or off through changes in Idrive. onoff can be string value 
    # 'on' or 'off', determining the action to be taken and bb_listtoswitch are the IDs of the backbones to apply this action to.
    global neurons
    
    if onoff == 'on': #Turns on backbone through higher Idrive.
        #print('bb',bb_listtoswitch, 'turned on')
        for nrn in neurons:
            if nrn.backbone_ID in bb_listtoswitch: #if the nrn belongs to a backbone in the list.
                nrn.Idrive = param.Idrive_min #This is the high Idrive value. Could use param.Idrive_max also because there is no distribution of E idrive.
                
    if onoff == 'off': #Turns off backbone through lower Idrive.
        #print('bb',bb_listtoswitch,'turned off')
        for nrn in neurons:
            if nrn.backbone_ID in bb_listtoswitch:
                nrn.Idrive = param.Idrive_NABB #Low enough to prevent backbone from theta spiking when it is "off."
                

                
def record_gaussian(): # Gives neuron objects their gaussian spike data. Must be run before any dot product functions can work.
    global neuron 
    
    for nrn in neurons: #This loop is to add the spike gaussians to the neurons.
        spike_gauss_sum = np.zeros(len(param.tarray)) #List to hold all the guassians together from one neuron.

        for spike_time in nrn.spikeTimes: #Note spikeTimes are still in ms/param.stepSize
            temp_gauss = stats.norm.pdf(param.tarray,loc=int(param.stepSize*spike_time),scale=2) #Temp list to hold 1 spike gaussian. Scale = 2 gives
            # Gaussian curve of total width about 10 ms. 
            spike_gauss_sum += temp_gauss #Adds to total sum.

        nrn.spike_gaussian = spike_gauss_sum #updates neuron object to the gaussian curves at each spike time.

        
        
    
    
def return_inhib_conns(nrn):
    # Function that returns the number of inhibitory connections to "nrn".
    count = 0
    for presyn,conn in enumerate(nrn.conn_in):
        if neurons[int(presyn)].backbone_ID == -1:
            count += conn # conn_in values are all 0 or 1, so summing gives the total in-degree from inhibitory neurons.
            
    return count #Returns number of inhibitory conns to this nrn. 
    



        
        
def assign_LE_Idrive(quad_colors): # Assigns new Idrives to LE neurons. quad_colors contains str of all
    # colors to change the Idrive for.
    global neuron
    
    for nrn in neurons:
        if nrn.backbone_ID == 0 and nrn.quad_color in quad_colors:
            nrn.Idrive = param.Idrive_LE # Assigns new Idrive.  
            

    
    
''' Need to change this function so that it handles multiple learning cycles'''  
def save_simdata(): # Function to save all useful information of the simulation data for the later creation of plots and 
    # measures.
    
    
    def export_LE_spikes(): # Separates LE spikes into each phase and writes them to 
        # txt file.

        LE_neurons = []
        for nrn in neurons:
            if nrn.backbone_ID == 0:
                LE_neurons.append(nrn)
        LE_neurons.sort(key=sort_LE_bycolor)

        all_spikes = [[] for i in range(len(LE_neurons))] # List for holding all spikes accross entire simulation
        pretest_BB1_spikes = [[] for i in range(len(LE_neurons))] # Pre-learning during BB1 activity
        pretest_BB2_spikes = [[] for i in range(len(LE_neurons))] # Pre-learning during BB2 activity
        NREM_spikes = [[] for i in range(len(LE_neurons))] # Holds spikes that occur in first phase of learning (usually NREM)
        NREMtest_BB1_spikes = [[] for i in range(len(LE_neurons))] # Post-NREM during BB1 activity
        NREMtest_BB2_spikes = [[] for i in range(len(LE_neurons))] # Post-NREM during BB1 activity
        REM_spikes = [[] for i in range(len(LE_neurons))] # Second phase
        posttest_BB1_spikes = [[] for i in range(len(LE_neurons))] # post-learning during BB1 activity
        posttest_BB2_spikes = [[] for i in range(len(LE_neurons))] # post-learning during BB2 activity


        for i,LE_nrn in enumerate(LE_neurons):

            if len(LE_nrn.spikeTimes) > 0: # So long as the neuron actually spikes

                for spike_t in LE_nrn.spikeTimes*param.stepSize: # NOTE nrn.spikeTimes gives times in ms/param.stepSize, so we have to get back
                    # to ms. spike_t is now in ms.
                    all_spikes[i].append(spike_t)

                    if param.plas_skip_time+20 < spike_t < param.BB_len_test: # For BB1 spikes in pre-learning. Skips strange synchronous burst
                        # that appears at start of every simulation.
                        pretest_BB1_spikes[i].append(spike_t)

                    if param.BB_len_test < spike_t < param.t_start_NREM: # For BB2 spikes in pre-learning
                        pretest_BB2_spikes[i].append(spike_t)

                    if param.t_start_NREM < spike_t < param.t_start_NREMtest: # For spikes in first phase (NREM)
                        NREM_spikes[i].append(spike_t)

                    if param.t_start_NREMtest < spike_t < param.t_start_NREMtest+param.BB_len_test: # For BB1 spikes in post-NREM test
                        NREMtest_BB1_spikes[i].append(spike_t)

                    if param.t_start_NREMtest+param.BB_len_test < spike_t < param.t_start_REM: # For BB2 spikes in post-NREM test
                        NREMtest_BB2_spikes[i].append(spike_t)

                    if param.t_start_REM < spike_t < param.t_start_posttest: # For spikes in second phase (REM)
                        REM_spikes[i].append(spike_t)

                    if param.t_start_posttest < spike_t < param.t_start_posttest+param.BB_len_test: # For BB1 spikes in post-learning
                        posttest_BB1_spikes[i].append(spike_t)

                    if param.t_start_posttest+param.BB_len_test < spike_t: # For BB2 spikes in post-learning
                        posttest_BB2_spikes[i].append(spike_t)


        # NOTE that opening a file in the write mode (using "w") automatically clears the file when it is opened, so I am always 
        # writing onto an empty file.
        with open(param.directory+"all_LE_spikes.txt", "w") as output:
            output.write(str(all_spikes))
        with open(param.directory+"pretest_BB1_LE_spikes.txt", "w") as output:
            output.write(str(pretest_BB1_spikes))
        with open(param.directory+"pretest_BB2_LE_spikes.txt", "w") as output:
            output.write(str(pretest_BB2_spikes))
        with open(param.directory+"NREM_LE_spikes.txt", "w") as output:
            output.write(str(NREM_spikes))
        with open(param.directory+"NREMtest_BB1_LE_spikes.txt", "w") as output:
            output.write(str(NREMtest_BB1_spikes))
        with open(param.directory+"NREMtest_BB2_LE_spikes.txt", "w") as output:
            output.write(str(NREMtest_BB2_spikes))
        with open(param.directory+"REM_LE_spikes.txt", "w") as output:
            output.write(str(REM_spikes))
        with open(param.directory+"posttest_BB1_LE_spikes.txt", "w") as output:
            output.write(str(posttest_BB1_spikes))
        with open(param.directory+"posttest_BB2_LE_spikes.txt", "w") as output:
            output.write(str(posttest_BB2_spikes))
            
    
#     export_LE_spikes() # Runs the function so that LE spike files are saved.
        
        
    
    
    def save_object(obj, filename): # For saving objects in .pkl files.
        with open(filename, 'wb') as outp:  # Overwrites any existing file.
            pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)
    
    
    save_object(neurons, param.directory+"neuron_objects.pkl") # Saves all neuron objects to a pickle file, which can be reimported to a 
    # notebook later! 
    
    save_object(param, param.directory+"param_object.pkl") # Saves param object for reimportation to another notebook.
    


In [3]:
param = init_param() # Initializes parameters
neurons,nc_Matrix = init_nrn() #initializes neurons and creates universal list.
strengthen_backbone()
init_quad_colors() # Groups LE neurons based on intial connectivities to bbs.
assign_LE_Idrive(['purple','pink'])
strengthen_LE() #Must come after init_quad_colors().

def mainProgramLoop():
    
    for t_ind in tqdm(range(param.Ntimes)):
        
        #Records timing of spikes (in t/stepSize)
        updateSpikeTime(t_ind)
        #Updates the input synaptic current to be used in RK4
        updateSyn(t_ind)
        #A function to update the solutions for all neurons' D.E.s
        RK4(t_ind)

        vary_param(t_ind) #Checks t_ind to change network gks values. Also added backbone switching and plasticity.
        
        zeroTempVars() #Resets temporary variables like Isyn
        
    
    return 

mainProgramLoop()

record_gaussian()

save_simdata()

100%|████████████████████████████████████████████████████████████████████████████| 70000/70000 [59:27<00:00, 19.62it/s]
